In [ ]:
#!pip install tensorflow==2.11.0 tensorflow-io==0.31.0 matplotlib pydub --quiet

In [ ]:
# Import necessary libraries
from pathlib import Path
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from matplotlib import pyplot as plt

# Define constants
FRAME_LENGTH = 320
FRAME_STEP = 32
MAX_LENGTH = 48000
BATCH_SIZE = 8

In [ ]:
# Function to normalize audio
def normalize_audio(audio):
    max_amplitude = tf.reduce_max(tf.abs(audio))
    normalized_audio = audio / max_amplitude
    return normalized_audio

# Function to load WAV file and normalize audio
def load_wav_16k_mono(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return normalize_audio(wav)

# Function for dynamic padding
def dynamic_padding(audio, max_length=MAX_LENGTH):
    audio_length = tf.shape(audio)[0]
    padding = tf.maximum(0, max_length - audio_length)
    audio = tf.pad(audio, [[0, padding]])
    return audio[:max_length]

# Function for preprocessing
def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    wav = dynamic_padding(wav)
    spectrogram = tf.signal.stft(wav, frame_length=FRAME_LENGTH, frame_step=FRAME_STEP)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [ ]:
# Define paths using pathlib
Filepath_TS2_4 = Path('data') / '2_4'
Filepath_TS3_4 = Path('data') / '3_4'
Filepath_TS4_4 = Path('data') / '4_4'

# Create TensorFlow datasets
Dataset_TS2_4 = tf.data.Dataset.list_files(str(Filepath_TS2_4 / '*.wav'))
Dataset_TS3_4 = tf.data.Dataset.list_files(str(Filepath_TS3_4 / '*.wav'))
Dataset_TS4_4 = tf.data.Dataset.list_files(str(Filepath_TS4_4 / '*.wav'))

# Create labels for each dataset
labels_2_4 = tf.constant([0] * len(Dataset_TS2_4))
labels_3_4 = tf.constant([1] * len(Dataset_TS3_4))
labels_4_4 = tf.constant([2] * len(Dataset_TS4_4))

# One-hot encode the labels
labels_2_4 = to_categorical(labels_2_4, num_classes=3)
labels_3_4 = to_categorical(labels_3_4, num_classes=3)
labels_4_4 = to_categorical(labels_4_4, num_classes=3)

# Create the datasets
TS2_4 = tf.data.Dataset.zip((Dataset_TS2_4, tf.data.Dataset.from_tensor_slices(labels_2_4)))
TS3_4 = tf.data.Dataset.zip((Dataset_TS3_4, tf.data.Dataset.from_tensor_slices(labels_3_4)))
TS4_4 = tf.data.Dataset.zip((Dataset_TS4_4, tf.data.Dataset.from_tensor_slices(labels_4_4)))

# Concatenate the datasets
data_all = TS2_4.concatenate(TS3_4).concatenate(TS4_4)

In [ ]:
# Create TensorFlow data pipeline
data = data_all.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(BATCH_SIZE)
data = data.prefetch(2)

# Split into training and testing partitions
total_samples = data.cardinality().numpy()
train_ratio = 0.7
test_ratio = 0.3
train_size = int(train_ratio * total_samples)
test_size = int(test_ratio * total_samples)
train_data = data.take(train_size)
test_data = data.skip(train_size).take(test_size)

In [ ]:
# samples, labels = train_data.as_numpy_iterator().next()
# samples.shape

In [ ]:
# Build deep learning model
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(1491, 257, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

model.summary()

In [ ]:
# Fit the model
hist = model.fit(train_data, epochs=10, validation_data=test_data)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 8))

# Plot Loss
plt.subplot(2, 2, 1)
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy
plt.subplot(2, 2, 2)
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Precision
plt.subplot(2, 2, 3)
plt.plot(hist.history['precision'], label='Training Precision')
plt.plot(hist.history['val_precision'], label='Validation Precision')
plt.title('Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# Plot Recall
plt.subplot(2, 2, 4)
plt.plot(hist.history['recall'], label='Training Recall')
plt.plot(hist.history['val_recall'], label='Validation Recall')
plt.title('Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Save the model
model.save(r'C:\Users\bianc\Desktop')